In [ ]:
#| default_exp utilities.nbdev

# utilities.nbdev

> Extra functionality for [nbdev](https://nbdev.fast.ai/)

In [ ]:
#| hide
from fastcore.test import test_eq

### DictTbl -

Based on nbdev `DocmentTbl`.

In [ ]:
#| export
import inspect
from fastcore.foundation import L

In [ ]:
#| export
def _split_at(s:str, *args, strip=False):
    result = [s]
    for arg in args:
        strs = result.pop().split(arg)
        if len(strs) == 2: result += strs
        else: return None
    if strip: result = [s.strip(strip) for s in result]
    return result        

In [ ]:
#| export
def _bold(s): return f'**{s}**' if s.strip() else s

In [ ]:
#|export
def _list2row(l:list): return '| '+' | '.join(l) + ' |'

In [ ]:
#| export
class MarkdownTable:
    
    def __init__(self, data, headers):
        self.data = data
        self.headers = headers
    
    @property
    def _columns(self):
        return self._map

    @property
    def _hdr_list(self): return [_bold(l) for l in self.headers]

    @property
    def hdr_str(self):
        "The markdown string for the header portion of the table"
        md = _list2row(self._hdr_list)
        return md + '\n' + _list2row(['-' * len(l) for l in self._hdr_list])

    @property
    def params_str(self):
        "The markdown string for the parameters portion of the table."
        return '\n'.join(L(self.data).map(_list2row))

    def _repr_markdown_(self):
        _tbl = [self.hdr_str, self.params_str]
        return '\n'.join(_tbl)

    def __eq__(self,other): return self.__str__() == str(other).strip()

    __str__ = _repr_markdown_

In [ ]:
#| export
class DictTbl(MarkdownTable):
    def __init__(self, obj):
        rows = []
        for s in inspect.getsource(obj).split('\n'):
            line = L(_split_at(s, ":", "#", strip=' '))
            if len(line) == 3 and line[0][0] in "\'\"": rows.append(s.strip('\"\'') for s in line[[0, -1]])
        super().__init__(rows, [' ', 'Description'])

In [ ]:
def _test_dict(): 
    return {
        'foo': [1, 2], # First item description
        'bar': [3, 4], # Item description No. 2
        'baz': [5, 6], # The last description
    }

In [ ]:
DictTbl(_test_dict)

|   | **Description** |
| - | --------------- |
| foo | First item description |
| bar | Item description No. 2 |
| baz | The last description |

In [ ]:
str(DictTbl(_test_dict))

'|   | **Description** |\n| - | --------------- |\n| foo | First item description |\n| bar | Item description No. 2 |\n| baz | The last description |'

In [ ]:
#| hide
test_eq(
    _, 
    '|   | **Description** |\n| - | --------------- |\n| foo | First item description |\n| bar | Item description No. 2 |\n| baz | The last description |'
)

### AttributeTbl -

In [ ]:
#| export
class AttributeTbl(MarkdownTable):
    def __init__(self, obj):
        rows = []
        for s in inspect.getsource(obj.__init__).split('\n'):
            line = L(_split_at(s, " = ", " # ", strip=' '))
            if len(line) == 3 and line[0].startswith('self.'): rows.append([line[0][5:], line[2]])
        super().__init__(rows, ['Attribute', 'Description'])

In [ ]:
class _test_class:
    """A test class for the AttributeTbl class."""
    
    class_variable = 'foo'
    
    def __init__(
        self, 
        bar:str, # bar parameter
        baz # baz parameter
    ):
        a = 3
        self.b = 4 # An attribute
        self.bing = 5 # Another attribute
        c = 6

In [ ]:
AttributeTbl(_test_class)

| **Attribute** | **Description** |
| ------------- | --------------- |
| b | An attribute |
| bing | Another attribute |

In [ ]:
str(AttributeTbl(_test_class))

'| **Attribute** | **Description** |\n| ------------- | --------------- |\n| b | An attribute |\n| bing | Another attribute |'

In [ ]:
#| hide
test_eq(
    _, 
    str(AttributeTbl(_test_class))
)

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()